# Uniswap V3 price analysis

In this notebook we will show how to download price events from Uniswap V3 to your computer as CSV files and use them to analyse price in each pool.

You will need to havev Ethereum JSON-RPC API node URL in order to pull out the data from Ethereum blockchain.
You will be interactively asked for this API key.

### Setup the notebook

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd
import matplotlib.pyplot as plt

from eth_defi.uniswap_v3.constants import UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
from eth_defi.uniswap_v3.events import fetch_events_to_csv

### Download the raw data

For simplicity, you can sign up for free access to an Ethereum node for example at [Infura](https://infura.io/), however we recommend you to [run your own Ethereum node](https://tradingstrategy.ai/blog/preparing-a-server-for-hosting-goethereum-full-node)/

In [7]:
# Get your node JSON-RPC URL
json_rpc_url = input("Please enter your Ethereum mainnet JSON-RPC URL here")

As an example, here we download raw events from first 1000000 blocks after Uniswap V3 factory is created, the events will be stored in several CSV files at `/tmp` folder.

The scan will take hours. It can resumed from crash, as we save the last scanned block in a JSON state file.

In [8]:
from eth_defi.event_reader.json_state import JSONFileScanState

start_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
end_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK + 1_000_000

state = JSONFileScanState("/tmp/price-scan.json")

fetch_events_to_csv(json_rpc_url, state, start_block=start_block, end_block=end_block)

Starting to read block range 12,369,621 - 13,369,621


Block: 12,371,121, events: 79, time:05-05-2021:   0%|          | 1600/1000000 [00:14<2:38:38, 104.90it/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: vitalik.tradingstrategy.ai. Connection pool size: 10
Block: 12,375,921, events: 314, time:05-05-2021:   1%|          | 6400/1000000 [00:32<1:29:25, 185.17it/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: vitalik.tradingstrategy.ai. Connection pool size: 10
Block: 12,376,021, events: 516, time:05-05-2021:   1%|          | 6500/1000000 [00:36<3:12:11, 86.15it/s] WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: vitalik.tradingstrategy.ai. Connection pool size: 10
Block: 12,379,321, events: 12139, time:06-05-2021:   1%|          | 9800/1000000 [01:32<2:35:28, 106.15it/s]

KeyboardInterrupt



### Analyse the pool's price

In Uniswap V3, you can get the price of any pool from any given moment using only swap events

In [ ]:
swap_df = pd.read_csv("/tmp/uniswapv3-Swap.csv")

Choose a pool to analyse, for example: USDC/ETH 0.3%

In [ ]:
pool_address = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"

Extract only swaps related to this pool from the dataframe

In [ ]:
df = swap_df[swap_df.pool_contract_address == pool_address]
df

In Uniswap V3, we can easily get pool's price from pool's tick

In [ ]:
def tick_to_price(tick):
    return 1.0001**tick

df["price"] = df.apply(lambda row: tick_to_price(row["tick"]), axis=1)

Then we can plot the price of this pool in first 1000000 blocks

In [ ]:
plt.rcParams["figure.dpi"] = 200

df.index = df.timestamp
df["price"].plot()

plt.xticks(rotation=45, ha="right")